# Sem Tech Project Gruppe 1

## Define Company Object Class

In [1]:
class CompanyObject:
    def __init__(self, list_Languages, list_Lables, list_Aliases, isSubsidiaryOf="None",
                 TwitterUsernames="", TwitterIDs=0, compID=0):
        
        self.compID=compID
        self.list_Languages=list_Languages
        self.list_Lables=list_Lables
        self.list_Aliases=list_Aliases
        self.isSubsidiaryOf=isSubsidiaryOf
        self.TwitterUsernames=TwitterUsernames
        self.TwitterIDs=TwitterIDs
    
    def __str__(self):
        s="ID: "+str(self.compID)+" with "+str(len(self.list_Languages))+" languages and TwitterUsernames: "+str(self.TwitterUsernames)
        if(self.isSubsidiaryOf != "null"):
            s+=" with Subsidiary: "+str(self.isSubsidiaryOf.compID)
        return s

# Crawl HTML Code

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


### First Tests of HTML Extraction

def AddSocialMediaToCompany(comp):
    response = requests.get(url=("https://www.wikidata.org/wiki/"+comp.compID))
    soup = BeautifulSoup(response.content, 'html.parser')
    http = soup.find_all('a', {'href':re.compile('https://twitter.com')})
    nameList=[]
    idList=[]
    for i in range(0, len(http)):
        url=http[i]['href'].rsplit('/', 1)[-1]
        if(url.isdigit()):
            idList.append(url)
        else:
            nameList.append(url)
    comp.TwitterUsernames=nameList
    comp.TwitterIDs=idList
    
    
response = requests.get(
	url="https://www.wikidata.org/wiki/Q20716",
)
soup = BeautifulSoup(response.content, 'html.parser')


twittername = soup.find_all("div", id=re.compile("Q20716\$"))
                            
http = soup.find_all('a', {'href':re.compile('https://twitter.com')})

#print(http)
#url = "https://twitter.com/i/user/2381578122"
#print(url.rsplit('/', 1)[-1])
#twitterNameList = [http[i] for i in range(0, len(http), 2)]

nameList=[]
idList=[]

#userid = [id for in http]
#print(http[0]['href'])

for i in range(0, len(http)):
    url=http[i]['href'].rsplit('/', 1)[-1]
    if(url.isdigit()):
        idList.append(url)
    else:
        nameList.append(url)
#print(idList,nameList)



# Crawl Json

In [3]:
import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

def isNaN(num):
    return num!= num

def getDaughterID(comp):
    return comp['besteht_aus']['value'].rsplit('/', 1)[-1]

def crawl(compID, compMotherObject="null"):
    response = requests.get(
        url=("https://www.wikidata.org/wiki/Special:EntityData/"+compID+".json"))
    data = json.loads(response.text)
    df = pd.DataFrame(data['entities'][compID]['labels'])
    
    languages=df.iloc[0]
    labels=df.iloc[1]
    
    df = pd.DataFrame(data['entities'][compID])
    
    temp = [i for i in df['aliases'] if not (isNaN(i))]
    
    aliases=[]

    for i in temp:
        for a in i:
            aliases.append(a)
    
    # Example Accesses
    #print(new,"ENDE")
    #print(new[0]['language'])
    #print(new[0]['value'])

    
### TODO: Get further Data from HTML
    
     
    compObject = CompanyObject(languages, labels, aliases, compID=compID,
                               TwitterUsernames="Unknown", isSubsidiaryOf=compMotherObject
                             )

    AddSocialMediaToCompany(compObject)
    print(compObject)
    
    return compObject

#crawl('Q20716')

# Länge der Liste
#print(len(results["results"]["bindings"]))

#for result in results["results"]["bindings"][:10]:
#    pass

## Get whole Company List

In [98]:
# pip install sparqlwrapper
# pip install pandas
# https://rdflib.github.io/sparqlwrapper/

import sys
import time
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

### Die aktuelle Query, welche zu jeder Company auch noch die SubCompanies liefert

query="""SELECT ?company ?besteht_aus WHERE {
  ?company wdt:P31 wd:Q4830453.
  OPTIONAL { ?company wdt:P1830 ?besteht_aus. }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# Länge der Liste

size=len(results["results"]["bindings"])
#print(size)

#print(results["results"]["bindings"][:10])

#liste = results["results"]["bindings"][:10]

#print(liste[0])

wholeCompList=[]
#for i in range(0,size)
#for i in range(0,100):
i=0








### Change number of Iterations here:

while(i<500):
    print("----------------",i,"------------------------")
    id=results["results"]["bindings"][i]['company']['value'].rsplit('/', 1)[-1]
    

    
    list_tovisit=[]
    try:
        list_tovisit.append(results["results"]["bindings"][i]['besteht_aus']['value'].rsplit('/', 1)[-1])
    except:
        pass
    
    for j in range(i+1,size):
        nextElem=results["results"]["bindings"][j]
        if (nextElem['company']['value'].rsplit('/', 1)[-1] == id):
            list_tovisit.append(getDaughterID(nextElem))
        else:
            i=j
            break;
            
    company = crawl(id)
    wholeCompList.append(company)
    for e in list_tovisit:
        print("................................")
        wholeCompList.append(crawl(e,company))
       # time.sleep(2)
    
for comp in wholeCompList:
    ### Do Whatever you want ;)
    pass
print("We Are Done and Happy")

---------------- 0 ------------------------
ID: Q66 with 109 languages and TwitterUsernames: ['Boeing']
................................
ID: Q8793 with 47 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q374451 with 23 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q570930 with 17 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q795324 with 29 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q890212 with 23 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q961568 with 11 languages and TwitterUsernames: [] with Subsidiary: Q66
................................
ID: Q1236833 with 37 languages and TwitterUsernames: ['ulalaunch'] with Subsidiary: Q66
................................
ID: Q1541150 with 12 languages and TwitterUsernames: [] with Subsidiary: Q66
.............

## -------------------------------------------------------------

# Setting up the ontology

In [129]:
from owlready2 import *

#load the basic structure of the ontology that was created in webprotege
onto = get_ontology("file://names_onto.owl").load()

classes = []

#extract the classes so we can create instances
for cl in onto.classes():
    classes.append(cl)



# Basic functions to create instances in order to fill our ontology

In [6]:
def get_Company_Instance(id):
    cl = [c for c in classes if c.name == "Company"]
    return cl[0](id)

def get_Alias_Instance(id):
    cl = [c for c in classes if c.name == "Alias"]
    return cl[0](id)

def get_Language_Instance(id):
    cl = [c for c in classes if c.name == "Language"]
    return cl[0](id)

def get_Dialect_Instance(id):
    cl = [c for c in classes if c.name == "Dialect"]
    return cl[0](id)

def get_Name_Instance(id, name):
    cl = [c for c in classes if c.name == "Name"]
    instance = cl[0](id)
    instance.Name = name
    return instance

def get_TikTok_Instance(id):
    cl = [c for c in classes if c.name == "TikTik_Username"]
    return cl[0](id)

def get_Twitter_Instance(id):
    cl = [c for c in classes if c.name == "Twitter_Username"]
    return cl[0](id)
    
def get_Youtube_Instance(id):
    cl = [c for c in classes if c.name == "Youtube_Username"]
    return cl[0](id)

def get_SubCompany_Instance(id):
    cl = [c for c in classes if c.name == "Subsidiary_company"]
    return cl[0](id)


# Filling the Ontology

In [109]:
languages_str = []
languages_onto = []
companies = []
owned_by = []
names_str = []
for comp in wholeCompList:
    id = comp.compID

    aliases = []
    for alias in comp.list_Aliases:
        lang = alias["language"]
        if(lang not in languages_str):
            languages_str.append(lang)
            lang_instance = get_Language_Instance(lang)
            languages_onto.append(lang_instance)
            al = get_Alias_Instance(alias["value"])
            al.is_word_in.append(lang_instance)
            aliases.append(al)
        else:
            lang_instance = [l for l in languages_onto if l.name == lang][0]
            al = get_Alias_Instance(alias["value"])
            al.is_word_in.append(lang_instance)
            aliases.append(al)
        
    names_onto_instances = []
    for k in comp.list_Lables.keys():
        names_str.append(comp.list_Lables.get(k))
        name = get_Name_Instance(comp.list_Lables.get(k))
        if(k not in languages_str): 
            languages_str.append(k)
            lang = get_Language_Instance(k)
            languages_onto.append(lang)
            name.is_word_in.append(lang)
        else:
            name.is_word_in.append([l for l in languages_onto if l.name == k][0])
        names_onto_instances.append(name)

    twitter_account_Usernames = []
    twitter_account_Ids = []
    for account, username in zip(comp.TwitterIDs, comp.TwitterUsernames):
        acc = get_Twitter_Instance(account)
        if(username not in names_str):
            names_str.append(username)
            name = get_Name_Instance(username)
            acc.has_name.append(name)
            name.is_name_for.append(acc)
            twitter_account_Usernames.append(name)
        twitter_account_Ids.append(acc)


    if(comp.isSubsidiaryOf != "null"):
        company = get_SubCompany_Instance(id)
        owned_by.append((comp.isSubsidiaryOf.compID, id))   
        for alias in aliases:
            company.has_name.append(alias)
            alias.is_alias_for.append(company)
        for name in names_onto_instances:
            company.has_name.append(name)
            name.is_name_for.append(company)
        for twitter_acc in twitter_account_Ids:
            company.has_social_media_name.append(twitter_acc)
            twitter_acc.belongs_to.append(company)
        companies.append(company)
    else:
        company = get_Company_Instance(id)
        for alias in aliases:
            company.has_name.append(alias)
            alias.is_alias_for.append(company)
        for name in names_onto_instances:
            company.has_name.append(name)
            name.is_name_for.append(company)
        for twitter_acc in twitter_account_Ids:
            company.has_social_media_name.append(twitter_acc)
            twitter_acc.belongs_to.append(company)
        companies.append(company)

    
    

# Example on how to link created instances via properties

In [8]:

"""
alias = get_Alias_Instance("Alias")

Name = get_Name_Instance("Name")

alias.belongs_to.append(Name)
"""


'\nalias = get_Alias_Instance("Alias")\n\nName = get_Name_Instance("Name")\n\nalias.belongs_to.append(Name)\n'

# Some sample querries running on our ontology

In [128]:
print("Number of individuals:", list(default_world.sparql("""
           SELECT (COUNT(?x) AS ?nb)
           { ?x a owl:NamedIndividual . }
    """)))

print("One example class", list(default_world.sparql("""
           SELECT ?x
           { ?x rdfs:label "Alias" . }
    """)))

print("All classes with their sub-classes:", list((default_world.sparql("""
           SELECT ?x ?y
           { ?x a owl:Class . 
             ?x rdfs:subClassOf* ?y }
    """))))


Number of individuals: [[208379]]
One example class [[names_onto.Alias]]
All classes with their sub-classes: [[names_onto.Company, names_onto.Company], [names_onto.Company, owl.Thing], [names_onto.Name, names_onto.Name], [names_onto.Name, owl.Thing], [names_onto.Alias, names_onto.Alias], [names_onto.Alias, names_onto.Name], [names_onto.Alias, owl.Thing], [names_onto.Dialect, names_onto.Dialect], [names_onto.Dialect, names_onto.Language], [names_onto.Dialect, owl.Thing], [names_onto.Subsidiary_company, names_onto.Subsidiary_company], [names_onto.Subsidiary_company, names_onto.Company], [names_onto.Subsidiary_company, owl.Thing], [names_onto.Language, names_onto.Language], [names_onto.Language, owl.Thing], [names_onto.TikTok_Username, names_onto.TikTok_Username], [names_onto.TikTok_Username, names_onto.Name], [names_onto.TikTok_Username, owl.Thing], [names_onto.Twitter_Username, names_onto.Twitter_Username], [names_onto.Twitter_Username, names_onto.Name], [names_onto.Twitter_Username, ow

# Using sync_reasoner to find and fix inconsistencies 

In [10]:
#automatically finds inconsistencies and reclassifies individuals and superclasses
#especially helpful when circular dependencies are present 
#sync_reasoner()


#optional: use diffrent reasoner, no differences spotted yet
#sync_reasoner_pellet() 


#optional: also taking object property values into acount
#sync_reasoner(infer_property_values = True)

# Saving changes

In [110]:
# weird: owlready2 cannot overwrite a file passed into the function via the first parameter. It can only create a new files...

onto.save("filled_names_onto5", "rdfxml")